In [1]:
import pickle
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import time
import tensorflow as tf
import scipy.io.wavfile as wav
import numpy as np
from six.moves import xrange as range
from python_speech_features import mfcc

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
pickle_in = open("padsample.pickle","rb")
DATA = pickle.load(pickle_in)
m=0
for i in DATA:
    if m<i[0].shape[0]:
        m=i[0].shape[0]

In [3]:
m

1696

In [4]:
len(DATA)

5535

In [5]:

# Some configs
num_features = 13
# Accounting the 0th indice +  space + blank label = 28 characters
num_classes = 9
num_units=100

# Hyper-parameters
num_epochs = 2000
num_hidden = 50
num_layers = 1
batch_size = 8
initial_learning_rate = 1e-2
momentum = 0.8

num_examples = 32
num_batches_per_epoch = int(num_examples/batch_size)

inputs=np.array([np.array(list(a)+list(np.zeros([m-len(a),13]))) for a,b in DATA])
labels = np.array([np.array(b) for a,b in DATA])

In [6]:
def sparse_tuple_from(sequences, dtype=np.int32):
    indices = []
    values = []

    for n, seq in enumerate(sequences):
        indices.extend(zip([n]*len(seq), range(len(seq))))
        values.extend(seq)

    indices = np.asarray(indices, dtype=np.int64)
    values = np.asarray(values, dtype=dtype)
    shape = np.asarray([len(sequences), np.asarray(indices).max(0)[1]+1], dtype=np.int64)

    return indices, values, shape



In [7]:
def pad_sequences(sequences, maxlen=None, dtype=np.float32,
                  padding='post', truncating='post', value=0.):
    lengths = np.asarray([len(s) for s in sequences], dtype=np.int64)

    nb_samples = len(sequences)
    if maxlen is None:
        maxlen = np.max(lengths)

    # take the sample shape from the first non empty sequence
    # checking for consistency in the main loop below.
    sample_shape = tuple()
    for s in sequences:
        if len(s) > 0:
            sample_shape = np.asarray(s).shape[1:]
            break

    x = (np.ones((nb_samples, maxlen) + sample_shape) * value).astype(dtype)
    for idx, s in enumerate(sequences):
        if len(s) == 0:
            continue  # empty list was found
        if truncating == 'pre':
            trunc = s[-maxlen:]
        elif truncating == 'post':
            trunc = s[:maxlen]
        else:
            raise ValueError('Truncating type "%s" not understood' % truncating)

        # check `trunc` has expected shape
        trunc = np.asarray(trunc, dtype=dtype)
        if trunc.shape[1:] != sample_shape:
            raise ValueError('Shape of sample %s of sequence at position %s is different from expected shape %s' %
                             (trunc.shape[1:], idx, sample_shape))

        if padding == 'post':
            x[idx, :len(trunc)] = trunc
        elif padding == 'pre':
            x[idx, -len(trunc):] = trunc
        else:
            raise ValueError('Padding type "%s" not understood' % padding)
    return x, lengths

In [8]:
# You can preprocess the input data here
train_inputs = inputs

# You can preprocess the target data here
train_targets = labels


In [9]:
graph = tf.Graph()
with graph.as_default():
    # e.g: log filter bank or MFCC features
    # Has size [batch_size, max_stepsize, num_features], but the
    # batch_size and max_stepsize can vary along each step
    inputs = tf.placeholder(tf.float32, [None, None, num_features])

    # Here we use sparse_placeholder that will generate a
    # SparseTensor required by ctc_loss op.
    targets = tf.sparse_placeholder(tf.int32)

    # 1d array of size [batch_size]
    seq_len = tf.placeholder(tf.int32, [None])

    # Defining the cell
    # Can be:
    #   tf.nn.rnn_cell.RNNCell
    #   tf.nn.rnn_cell.GRUCell
    # Stacking rnn cells
    cells = []
    for _ in range(num_layers):
        cell = tf.contrib.rnn.LSTMCell(num_units)  # Or LSTMCell(num_units)
        cells.append(cell)
    stack = tf.contrib.rnn.MultiRNNCell(cells)

    # The second output is the last state and we will no use that
    outputs, _ = tf.nn.dynamic_rnn(stack, inputs, seq_len, dtype=tf.float32)

    shape = tf.shape(inputs)
    batch_s, max_timesteps = shape[0], shape[1]

    # Reshaping to apply the same weights over the timesteps
    outputs = tf.reshape(outputs, [-1, num_hidden])

    # Truncated normal with mean 0 and stdev=0.1
    # Tip: Try another initialization
    # see https://www.tensorflow.org/versions/r0.9/api_docs/python/contrib.layers.html#initializers
    W = tf.Variable(tf.truncated_normal([num_hidden,
                                         num_classes],
                                        stddev=0.1))
    # Zero initialization
    # Tip: Is tf.zeros_initializer the same?
    b = tf.Variable(tf.constant(0., shape=[num_classes]))

    # Doing the affine projection
    logits = tf.matmul(outputs, W) + b

    # Reshaping back to the original shape
    logits = tf.reshape(logits, [batch_s, -1, num_classes])

    # Time major
    logits = tf.transpose(logits, (1, 0, 2))

    loss = tf.nn.ctc_loss(targets, logits, seq_len)
    cost = tf.reduce_mean(loss)

    optimizer = tf.train.MomentumOptimizer(initial_learning_rate,
                                           0.9).minimize(cost)

    # Option 2: tf.nn.ctc_beam_search_decoder
    # (it's slower but you'll get better results)
    decoded, log_prob = tf.nn.ctc_greedy_decoder(logits, seq_len)

    # Inaccuracy: label error rate
    ler = tf.reduce_mean(tf.edit_distance(tf.cast(decoded[0], tf.int32),
                                          targets))
    saver = tf.train.Saver()


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [10]:

with tf.Session(graph=graph) as session:
    # Initializate the weights and biases
    tf.global_variables_initializer().run()


    for curr_epoch in range(num_epochs):
        train_cost = train_ler = 0
        start = time.time()

        for batch in range(num_batches_per_epoch):

            # Getting the index
            indexes = [i % num_examples for i in range(batch * batch_size, (batch + 1) * batch_size)]

            batch_train_inputs = train_inputs[indexes]
            # Padding input to max_time_step of this batch
            batch_train_inputs, batch_train_seq_len = pad_sequences(batch_train_inputs)

            # Converting to sparse representation so as to to feed SparseTensor input
            batch_train_targets = sparse_tuple_from(train_targets[indexes])

            feed = {inputs: batch_train_inputs,
                    targets: batch_train_targets,
                    seq_len: batch_train_seq_len}

            batch_cost, _ = session.run([cost, optimizer], feed)
            train_cost += batch_cost*batch_size
            train_ler += session.run(ler, feed_dict=feed)*batch_size


        # Shuffle the data
        shuffled_indexes = np.random.permutation(num_examples)
        train_inputs = train_inputs[shuffled_indexes]
        train_targets = train_targets[shuffled_indexes]

        # Metrics mean
        train_cost /= num_examples
        train_ler /= num_examples

        log = "Epoch {}/{}, train_cost = {:.3f}, train_ler = {:.3f}, time = {:.3f}"
        print(log.format(curr_epoch+1, num_epochs, train_cost, train_ler, time.time() - start))

    # Decoding all at once. Note that this isn't the best way

    # Padding input to max_time_step of this batch
    batch_train_inputs, batch_train_seq_len = pad_sequences(train_inputs)

    # Converting to sparse representation so as to to feed SparseTensor input
    batch_train_targets = sparse_tuple_from(train_targets)

    feed = {inputs: batch_train_inputs,
            targets: batch_train_targets,
            seq_len: batch_train_seq_len
            }
    save_path = saver.save(session, os.getcwd()+ "/tmp/model.ckpt")
    print("Model saved in path: %s" % save_path)

    # Decoding
    d = session.run(decoded[0], feed_dict=feed)
    dense_decoded = tf.sparse_tensor_to_dense(d, default_value=-1).eval(session=session)

    for i, seq in enumerate(dense_decoded):

        seq = [s for s in seq if s != -1]

        print('Sequence %d' % i)
        print('\t Original:\n%s' % train_targets[i])
        print('\t Decoded:\n%s' % seq)

Epoch 1/2000, train_cost = 1203.563, train_ler = 0.955, time = 2.167
Epoch 2/2000, train_cost = 388.069, train_ler = 1.029, time = 2.003
Epoch 3/2000, train_cost = 279.844, train_ler = 1.202, time = 2.033
Epoch 4/2000, train_cost = 380.494, train_ler = 0.835, time = 2.007
Epoch 5/2000, train_cost = 399.168, train_ler = 0.890, time = 1.994
Epoch 6/2000, train_cost = 298.292, train_ler = 1.320, time = 1.991
Epoch 7/2000, train_cost = 232.024, train_ler = 0.794, time = 2.000
Epoch 8/2000, train_cost = 262.222, train_ler = 0.867, time = 2.036
Epoch 9/2000, train_cost = 237.161, train_ler = 0.941, time = 2.036
Epoch 10/2000, train_cost = 188.056, train_ler = 0.804, time = 1.991
Epoch 11/2000, train_cost = 162.453, train_ler = 0.843, time = 1.996
Epoch 12/2000, train_cost = 144.832, train_ler = 0.598, time = 1.989
Epoch 13/2000, train_cost = 120.122, train_ler = 0.682, time = 2.000
Epoch 14/2000, train_cost = 124.774, train_ler = 0.622, time = 1.993
Epoch 15/2000, train_cost = 103.193, train

Epoch 122/2000, train_cost = 14.679, train_ler = 0.194, time = 2.061
Epoch 123/2000, train_cost = 14.640, train_ler = 0.181, time = 2.088
Epoch 124/2000, train_cost = 14.737, train_ler = 0.197, time = 2.040
Epoch 125/2000, train_cost = 14.498, train_ler = 0.178, time = 2.023
Epoch 126/2000, train_cost = 14.459, train_ler = 0.173, time = 2.025
Epoch 127/2000, train_cost = 14.390, train_ler = 0.191, time = 1.997
Epoch 128/2000, train_cost = 14.349, train_ler = 0.172, time = 2.138
Epoch 129/2000, train_cost = 14.329, train_ler = 0.193, time = 2.023
Epoch 130/2000, train_cost = 14.304, train_ler = 0.167, time = 2.048
Epoch 131/2000, train_cost = 14.187, train_ler = 0.185, time = 2.002
Epoch 132/2000, train_cost = 14.085, train_ler = 0.182, time = 2.078
Epoch 133/2000, train_cost = 14.125, train_ler = 0.182, time = 2.009
Epoch 134/2000, train_cost = 14.074, train_ler = 0.176, time = 2.016
Epoch 135/2000, train_cost = 14.027, train_ler = 0.177, time = 1.999
Epoch 136/2000, train_cost = 13.91

Epoch 241/2000, train_cost = 10.221, train_ler = 0.127, time = 2.017
Epoch 242/2000, train_cost = 10.273, train_ler = 0.124, time = 2.010
Epoch 243/2000, train_cost = 10.153, train_ler = 0.124, time = 2.048
Epoch 244/2000, train_cost = 10.208, train_ler = 0.124, time = 2.019
Epoch 245/2000, train_cost = 10.109, train_ler = 0.125, time = 2.007
Epoch 246/2000, train_cost = 10.096, train_ler = 0.127, time = 2.028
Epoch 247/2000, train_cost = 10.106, train_ler = 0.121, time = 2.013
Epoch 248/2000, train_cost = 10.012, train_ler = 0.124, time = 2.007
Epoch 249/2000, train_cost = 10.003, train_ler = 0.126, time = 2.014
Epoch 250/2000, train_cost = 9.952, train_ler = 0.115, time = 2.000
Epoch 251/2000, train_cost = 9.911, train_ler = 0.124, time = 2.013
Epoch 252/2000, train_cost = 9.860, train_ler = 0.119, time = 2.000
Epoch 253/2000, train_cost = 9.847, train_ler = 0.120, time = 1.996
Epoch 254/2000, train_cost = 9.811, train_ler = 0.125, time = 2.021
Epoch 255/2000, train_cost = 9.807, tra

Epoch 362/2000, train_cost = 7.808, train_ler = 0.105, time = 2.029
Epoch 363/2000, train_cost = 7.756, train_ler = 0.102, time = 2.059
Epoch 364/2000, train_cost = 7.800, train_ler = 0.102, time = 2.115
Epoch 365/2000, train_cost = 7.714, train_ler = 0.110, time = 2.101
Epoch 366/2000, train_cost = 7.699, train_ler = 0.105, time = 2.083
Epoch 367/2000, train_cost = 7.675, train_ler = 0.110, time = 2.082
Epoch 368/2000, train_cost = 7.656, train_ler = 0.105, time = 2.090
Epoch 369/2000, train_cost = 7.715, train_ler = 0.102, time = 2.054
Epoch 370/2000, train_cost = 7.653, train_ler = 0.105, time = 2.034
Epoch 371/2000, train_cost = 7.663, train_ler = 0.106, time = 2.071
Epoch 372/2000, train_cost = 7.605, train_ler = 0.105, time = 2.071
Epoch 373/2000, train_cost = 7.617, train_ler = 0.106, time = 2.145
Epoch 374/2000, train_cost = 7.666, train_ler = 0.104, time = 2.051
Epoch 375/2000, train_cost = 7.573, train_ler = 0.100, time = 2.032
Epoch 376/2000, train_cost = 7.554, train_ler = 

Epoch 483/2000, train_cost = 6.370, train_ler = 0.071, time = 2.026
Epoch 484/2000, train_cost = 6.422, train_ler = 0.073, time = 2.014
Epoch 485/2000, train_cost = 6.338, train_ler = 0.072, time = 2.008
Epoch 486/2000, train_cost = 6.353, train_ler = 0.068, time = 1.996
Epoch 487/2000, train_cost = 6.319, train_ler = 0.068, time = 2.013
Epoch 488/2000, train_cost = 6.315, train_ler = 0.067, time = 2.002
Epoch 489/2000, train_cost = 6.314, train_ler = 0.067, time = 2.015
Epoch 490/2000, train_cost = 6.296, train_ler = 0.069, time = 2.022
Epoch 491/2000, train_cost = 6.281, train_ler = 0.066, time = 2.014
Epoch 492/2000, train_cost = 6.270, train_ler = 0.066, time = 2.001
Epoch 493/2000, train_cost = 6.295, train_ler = 0.072, time = 2.016
Epoch 494/2000, train_cost = 6.302, train_ler = 0.067, time = 2.002
Epoch 495/2000, train_cost = 6.229, train_ler = 0.070, time = 2.018
Epoch 496/2000, train_cost = 6.245, train_ler = 0.068, time = 2.013
Epoch 497/2000, train_cost = 6.263, train_ler = 

...

Epoch 1443/2000, train_cost = 3.326, train_ler = 0.036, time = 2.030
Epoch 1444/2000, train_cost = 3.312, train_ler = 0.034, time = 1.999
Epoch 1445/2000, train_cost = 3.331, train_ler = 0.035, time = 1.998
Epoch 1446/2000, train_cost = 3.341, train_ler = 0.035, time = 1.997
Epoch 1447/2000, train_cost = 3.293, train_ler = 0.032, time = 2.019
Epoch 1448/2000, train_cost = 3.285, train_ler = 0.034, time = 2.013
Epoch 1449/2000, train_cost = 3.304, train_ler = 0.033, time = 2.008
Epoch 1450/2000, train_cost = 3.351, train_ler = 0.033, time = 2.020
Epoch 1451/2000, train_cost = 3.318, train_ler = 0.033, time = 2.080
Epoch 1452/2000, train_cost = 3.307, train_ler = 0.035, time = 2.016
Epoch 1453/2000, train_cost = 3.356, train_ler = 0.040, time = 2.050
Epoch 1454/2000, train_cost = 3.357, train_ler = 0.038, time = 2.017
Epoch 1455/2000, train_cost = 3.381, train_ler = 0.034, time = 2.049
Epoch 1456/2000, train_cost = 3.547, train_ler = 0.034, time = 2.019
Epoch 1457/2000, train_cost = 3.40

Epoch 1562/2000, train_cost = 3.088, train_ler = 0.036, time = 2.124
Epoch 1563/2000, train_cost = 3.192, train_ler = 0.036, time = 2.114
Epoch 1564/2000, train_cost = 3.096, train_ler = 0.038, time = 2.135
Epoch 1565/2000, train_cost = 3.062, train_ler = 0.035, time = 2.122
Epoch 1566/2000, train_cost = 3.069, train_ler = 0.035, time = 2.121
Epoch 1567/2000, train_cost = 3.054, train_ler = 0.035, time = 2.109
Epoch 1568/2000, train_cost = 3.044, train_ler = 0.035, time = 2.109
Epoch 1569/2000, train_cost = 3.026, train_ler = 0.036, time = 2.121
Epoch 1570/2000, train_cost = 3.030, train_ler = 0.035, time = 2.109
Epoch 1571/2000, train_cost = 3.087, train_ler = 0.039, time = 2.112
Epoch 1572/2000, train_cost = 3.045, train_ler = 0.036, time = 2.096
Epoch 1573/2000, train_cost = 3.025, train_ler = 0.036, time = 2.113
Epoch 1574/2000, train_cost = 3.080, train_ler = 0.043, time = 2.120
Epoch 1575/2000, train_cost = 3.047, train_ler = 0.039, time = 2.117
Epoch 1576/2000, train_cost = 3.08

Epoch 1681/2000, train_cost = 2.882, train_ler = 0.032, time = 2.065
Epoch 1682/2000, train_cost = 2.874, train_ler = 0.030, time = 2.068
Epoch 1683/2000, train_cost = 2.905, train_ler = 0.033, time = 2.075
Epoch 1684/2000, train_cost = 2.866, train_ler = 0.031, time = 2.094
Epoch 1685/2000, train_cost = 2.894, train_ler = 0.031, time = 2.084
Epoch 1686/2000, train_cost = 2.925, train_ler = 0.033, time = 2.064
Epoch 1687/2000, train_cost = 2.916, train_ler = 0.032, time = 2.066
Epoch 1688/2000, train_cost = 2.966, train_ler = 0.034, time = 2.057
Epoch 1689/2000, train_cost = 3.072, train_ler = 0.038, time = 2.073
Epoch 1690/2000, train_cost = 3.071, train_ler = 0.036, time = 2.075
Epoch 1691/2000, train_cost = 2.925, train_ler = 0.033, time = 2.041
Epoch 1692/2000, train_cost = 2.982, train_ler = 0.036, time = 2.063
Epoch 1693/2000, train_cost = 3.005, train_ler = 0.036, time = 2.034
Epoch 1694/2000, train_cost = 2.897, train_ler = 0.035, time = 2.073
Epoch 1695/2000, train_cost = 2.95

Epoch 1800/2000, train_cost = 2.764, train_ler = 0.038, time = 2.070
Epoch 1801/2000, train_cost = 2.800, train_ler = 0.028, time = 2.036
Epoch 1802/2000, train_cost = 2.791, train_ler = 0.029, time = 2.051
Epoch 1803/2000, train_cost = 2.757, train_ler = 0.027, time = 2.040
Epoch 1804/2000, train_cost = 2.742, train_ler = 0.030, time = 2.065
Epoch 1805/2000, train_cost = 2.758, train_ler = 0.031, time = 2.037
Epoch 1806/2000, train_cost = 2.728, train_ler = 0.029, time = 2.046
Epoch 1807/2000, train_cost = 2.777, train_ler = 0.031, time = 2.071
Epoch 1808/2000, train_cost = 2.736, train_ler = 0.030, time = 2.054
Epoch 1809/2000, train_cost = 2.763, train_ler = 0.029, time = 2.068
Epoch 1810/2000, train_cost = 2.800, train_ler = 0.031, time = 2.053
Epoch 1811/2000, train_cost = 2.773, train_ler = 0.035, time = 2.036
Epoch 1812/2000, train_cost = 2.797, train_ler = 0.030, time = 2.066
Epoch 1813/2000, train_cost = 2.747, train_ler = 0.033, time = 2.035
Epoch 1814/2000, train_cost = 2.76

Epoch 1919/2000, train_cost = 2.666, train_ler = 0.030, time = 2.177
Epoch 1920/2000, train_cost = 2.671, train_ler = 0.038, time = 2.177
Epoch 1921/2000, train_cost = 2.675, train_ler = 0.032, time = 2.182
Epoch 1922/2000, train_cost = 2.640, train_ler = 0.029, time = 2.162
Epoch 1923/2000, train_cost = 2.678, train_ler = 0.031, time = 2.179
Epoch 1924/2000, train_cost = 2.630, train_ler = 0.029, time = 2.165
Epoch 1925/2000, train_cost = 2.724, train_ler = 0.029, time = 2.170
Epoch 1926/2000, train_cost = 2.740, train_ler = 0.029, time = 2.168
Epoch 1927/2000, train_cost = 2.800, train_ler = 0.039, time = 2.169
Epoch 1928/2000, train_cost = 2.694, train_ler = 0.031, time = 2.175
Epoch 1929/2000, train_cost = 2.680, train_ler = 0.034, time = 2.179
Epoch 1930/2000, train_cost = 2.701, train_ler = 0.030, time = 2.173
Epoch 1931/2000, train_cost = 2.615, train_ler = 0.029, time = 2.168
Epoch 1932/2000, train_cost = 2.752, train_ler = 0.029, time = 2.180
Epoch 1933/2000, train_cost = 2.62